### PII Middlewear

In [ ]:
from langchain.agents import create_agent
from langchain.agents.middleware import PIIMiddleware

agent = create_agent(
    model="gpt-4o",
    tools=[],
    middleware=[
        PIIMiddleware("email", strategy="redact", apply_to_input=True),
        PIIMiddleware("credit_card", strategy="mask", apply_to_input=True),
    ],
)

agent.invoke({"messages": [{"role": "user", "content": "Send an email to john.de@gmail.com  with my credit card 4111 1111 1111 1111"}]})

### Custom PII

In [ ]:
from langchain.agents import create_agent
from langchain.agents.middleware import PIIMiddleware
import re   


# Method 1: Regex pattern string
agent1 = create_agent(
    model="gpt-4o",
    tools=[],
    middleware=[
        PIIMiddleware(
            "api_key",
            detector=r"sk-[a-zA-Z0-9]{32}",
            strategy="block",
        ),
    ],
)

agent1.invoke({"messages": [{"role": "user", "content": "My API key is sk-1234567890abcdef1234567890abcdef"}]})



In [ ]:
import re

# Method 2: Compiled regex pattern
agent2 = create_agent(
    model="gpt-4o",
    tools=[],
    middleware=[
        PIIMiddleware(
            "phone_number",
            detector=re.compile(r"\+?\d{1,3}[\s.-]?\d{3,4}[\s.-]?\d{4}"),
            strategy="mask",
        ),
    ],
)
agent2.invoke({"messages": [{"role": "user", "content": "Call me at +1-800-555-1234 or 800.555.5678."}]})   



In [ ]:
# Method 3: Custom detector function
def detect_ssn(content: str) -> list[dict[str, str | int]]:
    """Detect SSN with validation.

    Returns a list of dictionaries with 'text', 'start', and 'end' keys.
    """
    import re
    matches = []
    pattern = r"\d{3}-\d{2}-\d{4}"
    for match in re.finditer(pattern, content):
        ssn = match.group(0)
        # Validate: first 3 digits shouldn't be 000, 666, or 900-999
        first_three = int(ssn[:3])
        if first_three not in [0, 666] and not (900 <= first_three <= 999):
            matches.append({
                "text": ssn,
                "start": match.start(),
                "end": match.end(),
            })
    return matches

agent3 = create_agent(
    model="gpt-4o",
    tools=[],
    middleware=[
        PIIMiddleware(
            "ssn",
            detector=detect_ssn,
            strategy="hash",
        ),
    ],
)

agent3.invoke({"messages": [{"role": "user", "content": "My SSN is 123-45-6789 and 000-12-3456."}]})

In [ ]:
from langchain.agents import create_agent
from langchain.agents.middleware import HumanInTheLoopMiddleware
from langgraph.checkpoint.memory import InMemorySaver
from langchain.tools import tool

@tool
def read_email_tool(email_id: str) -> str:
    """Mock function to read an email by its ID."""
    return f"Email content for ID: {email_id}"
@tool
def send_email_tool(recipient: str, subject: str, body: str) -> str:
    """Mock function to send an email."""
    return f"Email sent to {recipient} with subject '{subject}'"




checkpointer = InMemorySaver()
# `thread_id` is a unique identifier for a given conversation.
config = {"configurable": {"thread_id": "1"}}

agent = create_agent(
    model="gpt-4o",
    tools=[read_email_tool, send_email_tool],
    checkpointer=checkpointer,
    middleware=[
        HumanInTheLoopMiddleware(
            interrupt_on={
                "send_email_tool": {
                    "allowed_decisions": ["approve", "edit", "reject"],
                },
                "read_email_tool": False,
            }
        ),
    ],
)

In [ ]:
response = agent.invoke(
    {"messages": [{"role": "user", "content": "Read my latest email and send a reply to John Doe saying I'll be late to the meeting."}]},
    config=config
)

print(response)

In [ ]:
import os
from getpass import getpass
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print(f"API Key: {OPENAI_API_KEY}")

# must enter API key
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY") or \
    getpass("Enter LangSmith API Key: ")

LANGCHAIN_API_KEY = os.getenv("LANGCHAIN_API_KEY")
print(f"LANGCHAIN_API_KEY Key: {LANGCHAIN_API_KEY}")

# below should not be changed
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
# you can change this as preferred
os.environ["LANGCHAIN_PROJECT"] = "Rakeshgarapati-middlewear-demo"


In [ ]:
from langchain.agents import create_agent
from langchain.agents.middleware import ToolCallLimitMiddleware
from langchain_core.tools import tool


@tool(parse_docstring=True)
def search_items(query: str) -> str:
    """Search for items in the store.

    Args:
        query: Search query to find items.

    Returns:
        List of matching items with IDs and prices.
    """
    return """Found 3 items:
    - ID: hdpn-001, Sony WH-1000XM5, $299.99, rating 4.5/5
    - ID: hdpn-002, Bose QuietComfort 45, $279.00, rating 4.7/5
    - ID: hdpn-003, Apple AirPods Max, $549.00, rating 4.8/5"""


@tool(parse_docstring=True)
def purchase_item(item_id: str) -> str:
    """Purchase an item by its ID.

    Args:
        item_id: The unique identifier of the item to purchase.

    Returns:
        Purchase confirmation.
    """
    # Simulate purchase
    return f"✓ Successfully purchased item {item_id}"


PROMPT = """
You are a shopping assistant. Help users search for and purchase items.
Use the search_items tool to find items and the purchase_item tool to purchase items.
Make sure to use the ratings of the items to make the best purchase decision.
"""

agent = create_agent(
    model="openai:gpt-4o",
    tools=[search_items, purchase_item],
    system_prompt=PROMPT,
    middleware=[
        ToolCallLimitMiddleware(
            tool_name="purchase_item",
            run_limit=1,  # Max 1 purchase per conversation turn
            thread_limit=2,  # Max 2 purchases total across all conversation turns
        )
    ],
)
agent.invoke({"messages": [{"role": "user", "content": "Find and buy me the best headphones under $300."}]})